In [ ]:
!pip install -r requirements.txt

### Imports

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import load_dataset

### Quantization

In [ ]:
# https://huggingface.co/docs/peft/main/en/developer_guides/quantization

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
# Load in the model
model_id = "nvidia/OpenMath-Mistral-7B-v0.1-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
quantized_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0", quantization_config=quantization_config)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token
tokenizer.padding_side = "right"

In [ ]:
model = prepare_model_for_kbit_training(quantized_model)

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, lora_config)

### Datahandling

In [ ]:
from datasets import load_dataset

In [ ]:
train_dataset = load_dataset('json', data_files="/content/merged_math_problems_train.json", split="train")
val_dataset = load_dataset('json', data_files="/content/merged_math_problems_test.json")['train']

In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
text_format = """### Question: {problem_statement} [Answer] : {solution}"""

def formatting_prompts_func(dataset):
    output_texts = []

    for idx in range(len(dataset['level'])):
        problem = dataset['problem'][idx]
        solution = dataset['solution'][idx]
        text = text_format.format(problem_statement=problem, solution=solution)
        output_texts.append(text)

    return output_texts

In [ ]:
response_template = " [Answer] :"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

In [ ]:
trainer.train()